In [6]:
#importing tensor flow and making a construction graph
import tensorflow as tf

# reset_graph()

x = tf.Variable(5)
y = tf.Variable(4)
f = x*x*y + y + 2

#execution phase where we run a session differebt ways
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

#Using the with block to run our session
init = tf.global_variables_initializer()

with tf.Session() as sess:
#     x.initializer.run()
#     y.initializer.run()
    init.run()
    res = f.eval()
    print(res)

#using interactive session. here we dont have to set it defualt as it is set like that by design
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

In [ ]:
#in case of different graphs within the construction phase 

#making variable and checking if its added to the main graph
x1 = tf.Variable(10)
x1.graph is tf.get_default_graph()

#addding node to another graph(non default)
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is tf.get_default_graph()

In [16]:
w = tf.constant(3)
x = w + 2
p = w + 4
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15
    
    
    
#values in the array are calculated just once
with tf.Session() as sess:
    y_val, z_val, p_val = sess.run([y, z, p])
    print(y_val)  # 10
    print(z_val)  # 15
    print(p_val)




10
15
10
15
7


In [23]:
import numpy as np
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

#with tensorflow and normal equation
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print('using tensor, theta is ', theta_value)
    
#Doing with numpy only    
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)


print('numpy theta ', theta_numpy)

#doing with scikitlearn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print('scikit learn', np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

using tensor, theta is  [[-3.6894890e+01]
 [ 4.3661433e-01]
 [ 9.4453208e-03]
 [-1.0704148e-01]
 [ 6.4345831e-01]
 [-3.9632569e-06]
 [-3.7880042e-03]
 [-4.2093179e-01]
 [-4.3400639e-01]]
numpy theta  [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]
scikit learn [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [30]:
#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# print(scaled_housing_data_plus_bias.mean(axis=0))
# print(scaled_housing_data_plus_bias.mean(axis=1))
# print(scaled_housing_data_plus_bias.mean())
# print(scaled_housing_data_plus_bias.shape)



tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    my = training_op.eval()
    
    
print(my)
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705
[[ 2.06855226e+00]
 [ 7.74153352e-01]
 [ 1.31169230e-01]
 [-1.18057005e-01]
 [ 1.64984047e-01]
 [ 7.34676723e-04]
 [-3.91941592e-02]
 [-8.61454606e-01]
 [-8.23589861e-01]]
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44081801e-04]
 [-3.91945131e-02]
 [-8.61356556e-01]
 [-8.23479712e-01]]


In [8]:
#using auto diff 

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m,n = housing.data.shape


#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#trying builtin gradeints function
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

#trying the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)



init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [61]:
#using placeholder for mini batch descents

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape


#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

def fetch_data(batch_num,  batch_size):
    start = batch_num * batch_size
    X_batch = scaled_housing_data_plus_bias[start:start + batch_size]
    y_batch = housing.target.reshape(-1, 1)[start:start + batch_size] 
    return X_batch, y_batch

#-----------random mini batches (book)------------
# def fetch_batch(epoch, batch_index, batch_size):
#     np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
#     indices = np.random.randint(m, size=batch_size)  # not shown
#     X_batch = scaled_housing_data_plus_bias[indices] # not shown
#     y_batch = housing.target.reshape(-1, 1)[indices] # not shown
#     return X_batch, y_batch
    
    
with tf.Session() as sess:
    
    sess.run(init)
#     saver.restore(sess, '/tmp/my_model_final.ckpt') #if we call this we will not call init funct
    for epoch in range(n_epochs): 
        for a in range(n_batches ):
            X_batch, y_batch = fetch_data(a, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  

        if epoch % 100 == 0:
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={X: X_batch, y: y_batch}))
        
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')      
            
    
print(best_theta)
  
    
#--_Restoring from saved files   

# with tf.Session() as sess:
#     saver.restore(sess, "/tmp/my_model_final.ckpt")
#     best_theta_restored = theta.eval() # not shown in the book

# best_theta_restored

Epoch 0 MSE = 0.14887354
Epoch 100 MSE = 0.124388054
Epoch 200 MSE = 0.124388054
Epoch 300 MSE = 0.124388054
Epoch 400 MSE = 0.124388054
Epoch 500 MSE = 0.124388054
Epoch 600 MSE = 0.124388054
Epoch 700 MSE = 0.124388054
Epoch 800 MSE = 0.124388054
Epoch 900 MSE = 0.124388054
[[ 2.002909  ]
 [ 0.78347117]
 [ 0.13612016]
 [-0.2301369 ]
 [ 0.27156723]
 [-0.00706378]
 [-0.01321159]
 [-0.8690103 ]
 [-0.8959996 ]]


In [6]:
#tesnsor board testing
import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

# scaled_housing_data_plus_bias 


housing = fetch_california_housing()
m, n = housing.data.shape


#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]



from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)




X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

learning_rate = 0.01
n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")

y_pred = tf.matmul(X, theta, name="predictions")

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


# def fetch_data(batch_num,  batch_size):
#     start = batch_num * batch_size
#     X_batch = scaled_housing_data_plus_bias[start:start + batch_size]
#     y_batch = housing.target.reshape(-1, 1)[start:start + batch_size] 
#     return X_batch, y_batch
 

  
    
def fetch_data( batch_index, epoch):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch


with tf.Session() as sess:
    
    sess.run(init)
    for epoch in range(n_epochs): 
        for batch_index in range(n_batches ):
            X_batch, y_batch = fetch_data(batch_index,epoch)
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()
    
print(best_theta)


file_writer.close()

print(error.op.name)

print(mse.op.name)

[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558535]
 [-0.26835832]
 [ 0.32982782]
 [ 0.00608358]
 [ 0.07052915]
 [-0.87988573]
 [-0.8634251 ]]
loss/sub
loss/mse


In [8]:
tf.reset_default_graph()


a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="agf")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
agf
param/a
param_1/a


In [37]:
#DRY technique where you dont repeat the cod3

#all these are nodes until the session is run
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1), seed = 25), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1), seed = 25), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

data = [[1,2,3 ],[4.,5,6],[ 7,8,9]]
data1 = [[1,4, 7,]]
init = tf.global_variables_initializer()


#session running
with tf.Session() as sess:
    sess.run(init)
    print('value is ', output.eval(feed_dict = {X: data1}))

value is  [[1.763833]]


In [97]:
#this is trying the DRY technique about dont repeat yourslef
tf.reset_default_graph()


from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

#for sharing variables, adding threshold as parameter(this is NOT attribute)
def relu(X, threshold):
    #with tf.name_scope("relu"):  
    with tf.variable_scope('relu', reuse = True):
        threshold = tf.get_variable("threshold")
#         if not hasattr(relu, 'threshold'):
#             print('aint got it')
#             relu.threshold = tf.Variable(0.0, name="threshold")
#         else:
#             print('got it')
#         print('shape of x is ', (X.get_shape()))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape, seed = 25), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
#     threshold = tf.Variable(0.0, name = "threshold")
    tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0)) 
    print('name of var is ', threshold.name)

    
# threshold = tf.Variable(0.0, name = 'threshold') #this is for passing as a parameter 
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

data = [[1,2,3 ],[4.,5,6],[ 7,8,9]]
data1 = [[1,4, 7,]]
init = tf.global_variables_initializer()

#so this summary scalar has to be a value 
op_summary = tf.summary.scalar('output', output[0][0])
#first parameter is the location and name of file saved. relu2 is the folder within is the file
file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())

#session running
with tf.Session() as sess:
    sess.run(init)
    summary_str = op_summary.eval(feed_dict={X: data1})
    file_writer.add_summary(summary_str)
    print('value is ', output.eval(feed_dict = {X: data1}))
    
file_writer.close()

name of var is  relu/threshold:0
value is  [[4.4095826]]


In [86]:
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
var1:0
my_scope/var2:0
my_scope/Add:0


In [123]:
#question number 12
# from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

from sklearn.datasets import make_moons

tf.reset_default_graph()

m_data, m_labels = make_moons(n_samples=1000, shuffle=True, noise=0.15, random_state = 42)
m, n = m_data.data.shape


#doing linear regression by using gradient descent
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# m_data = scaler.fit_transf0orm(m_data)
scaled_mdata_bias = np.c_[np.ones((m, 1)), m_data]

#dividing training and testing
test_percent = 0.2
test_data_l = int(m * test_percent)

X_test = scaled_mdata_bias[:test_data_l,:]
y_test = m_labels[:test_data_l].reshape(-1,1)
X_train  = scaled_mdata_bias[test_data_l:,:]
y_train = m_labels[test_data_l:].reshape(-1,1)


X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

n_epochs = 100
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_p1 = tf.matmul(X, theta, name="predictions")
y_proba = tf.math.sigmoid(y_p1)

#defining loss
loss = tf.losses.log_loss(y, y_proba)  # uses epsilon = 1e-7 by default
# epsilon = 1e-7  # to avoid an overflow when computing the log
# loss = -tf.reduce_mean(y * tf.log(y_proba + epsilon) + (1 - y) * tf.log(1 - y_proba + epsilon))

learning_rate = 0.1
#the following statements will update thetas
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

def fetch_data(batch_num,  batch_size):
    start = batch_num * batch_size
    X_batch = X_train[start:start + batch_size]
    y_batch = y_train.reshape(-1, 1)[start:start + batch_size] 
    return X_batch, y_batch
    
    
with tf.Session() as sess:
    
    sess.run(init)
#    saver.restore(sess, '/tmp/my_model_final.ckpt') #if we call this we will not call init funct
    for epoch in range(n_epochs): 
        for a in range(n_batches):
            X_batch, y_batch = fetch_data(a, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  
        
        loss_val = loss.eval({X: X_test, y: y_test})
        if epoch % 10 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
         

    y_proba_val = y_proba.eval(feed_dict={X: X_test, y: y_test})
        
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')      
            
    
print(y_proba_val)
  
    
# # #--_Restoring from saved files   

# # # with tf.Session() as sess:
# # #     saver.restore(sess, "/tmp/my_model_final.ckpt")
# # #     best_theta_restored = theta.eval() # not shown in the book




Epoch: 0 	Loss: 0.77865386
Epoch: 10 	Loss: 0.42376786
Epoch: 20 	Loss: 0.36148316
Epoch: 30 	Loss: 0.334556
Epoch: 40 	Loss: 0.31864986
Epoch: 50 	Loss: 0.3078734
Epoch: 60 	Loss: 0.30004933
Epoch: 70 	Loss: 0.29414245
Epoch: 80 	Loss: 0.28957146
Epoch: 90 	Loss: 0.28597236
[[0.15426323]
 [0.94885755]
 [0.8448819 ]
 [0.90223813]
 [0.05432093]
 [0.82605135]
 [0.7718914 ]
 [0.14245051]
 [0.96562463]
 [0.09861547]
 [0.94452167]
 [0.27798605]
 [0.6817656 ]
 [0.9272815 ]
 [0.94340783]
 [0.92186844]
 [0.01557541]
 [0.9569267 ]
 [0.9524775 ]
 [0.01884568]
 [0.38941675]
 [0.0442746 ]
 [0.03886279]
 [0.20702586]
 [0.05652633]
 [0.03643975]
 [0.18318489]
 [0.8412297 ]
 [0.9505526 ]
 [0.8375961 ]
 [0.02107379]
 [0.09269968]
 [0.7809271 ]
 [0.1758399 ]
 [0.09421247]
 [0.01358429]
 [0.91760385]
 [0.58951706]
 [0.927708  ]
 [0.66614383]
 [0.00717553]
 [0.29311228]
 [0.01583025]
 [0.8375609 ]
 [0.02338424]
 [0.04230824]
 [0.9786184 ]
 [0.58802825]
 [0.15673918]
 [0.9617834 ]
 [0.46835142]
 [0.976897

In [117]:
y_test.reshape(-1,1).shape

(200, 1)